In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
class Node:
    def __init__(self, gini, num_samples, num_samples_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

In [ ]:
class DecisionTree:

  def __init__(self, X, y, num_classes):
    self.X = X
    self.y = y
    self.num_classes = num_classes
    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y)

  def find_split(self, X, y, n_classes):
    """ Find split where children has lowest impurity possible
    in condition where the purity should also be less than the parent,
    if not, stop.
    """
    n_samples, n_features = X.shape
    if n_samples <= 1:
          return None, None
    
    #so it will not have any warning about "referenced before assignments"
    feature_ix, threshold = None, None
    
    # Count of each class in the current node.
    sample_per_class_parent = [np.sum(y == c) for c in range(n_classes)] #[2, 2]
    
    # Gini of parent node.
    best_gini = 1.0 - sum((n / n_samples) ** 2 for n in sample_per_class_parent)

    # Loop through all features.
    for feature in range(n_features):
        
        # Sort data along selected feature.
        sample_sorted = sorted(X[:, feature]) #[2, 3, 10, 19]
        sort_idx = np.argsort(X[:, feature])
        y_sorted = y[sort_idx] #[0, 0, 1, 1]
                
        sample_per_class_left = [0] * n_classes   #[0, 0]
        
        sample_per_class_right = sample_per_class_parent.copy() #[2, 2]
        
        #loop through each threshold, 2.5, 6.5, 14.5
        #1st iter: [-] [-++]
        #2nd iter: [--] [++]
        #3rd iter: [--+] [+]
        for i in range(1, n_samples): #1 to 3 (excluding 4)
            #the class of that sample
            c = y_sorted[i - 1]  #[0]
            
            #put the sample to the left
            sample_per_class_left[c] += 1  #[1, 0]
                        
            #take the sample out from the right  [1, 2]
            sample_per_class_right[c] -= 1
            
            gini_left = 1.0 - sum(
                (sample_per_class_left[x] / i) ** 2 for x in range(n_classes)
            )
                        
            #we divided by n_samples - i since we know that the left amount of samples
            #since left side has already i samples
            gini_right = 1.0 - sum(
                (sample_per_class_right[x] / (n_samples - i)) ** 2 for x in range(n_classes)
            )

            #weighted gini
            weighted_gini = ((i / n_samples) * gini_left) + ( (n_samples - i) /n_samples) * gini_right

            # in case the value are the same, we do not split
            # (both have to end up on the same side of a split).
            if sample_sorted[i] == sample_sorted[i - 1]:
                continue
            if weighted_gini < best_gini:
                
                best_gini = weighted_gini
                feature_ix = feature
                threshold = (sample_sorted[i] + sample_sorted[i - 1]) / 2  # midpoint

      #return the feature number and threshold 
      #used to find best split
    return feature_ix, threshold

  def fit(self, Xtrain, ytrain, n_classes, max_depth, depth=0):  
      n_samples, n_features = Xtrain.shape
      num_samples_per_class = [np.sum(ytrain == i) for i in range(n_classes)]
      #predicted class using the majority of sample class
      predicted_class = np.argmax(num_samples_per_class)
      
      #define the parent node
      node = Node(
          gini = 1 - sum((np.sum(y == c) / n_samples) ** 2 for c in range(n_classes)),
          predicted_class=predicted_class,
          num_samples = ytrain.size,
          num_samples_per_class = num_samples_per_class,
          )

      #perform recursion
      feature, threshold = self.find_split(Xtrain, ytrain, n_classes)
      if depth < max_depth :
          #take all the indices that is less than threshold
        if feature is not None :
          indices_left = X[:, feature] < threshold
          X_left, y_left = X[indices_left], y[indices_left]
          X_right, y_right = X[~indices_left], y[~indices_left]
          node.feature_index = feature
          node.threshold = threshold
          node.left = self.fit(X_left, y_left, n_classes, max_depth, depth + 1)
          node.right = self.fit(X_right, y_right, n_classes, max_depth, depth + 1)
      return node

  #to predict, it is as simple as moving 
  #through the tree 
  def predict(self, sample, tree):
      while tree.left:
          if sample[tree.feature_index] < tree.threshold:
              tree = tree.left
          else:
              tree = tree.right
      return tree.predicted_class

  def output(self, max_depth):
    tree = self.fit(self.X_train, self.y_train, self.num_classes, max_depth)
    prediction = [self.predict(x, tree) for x in exp.X_test]
    return prediction

In [ ]:
a = np.array([1, 2, 1, 2])
b = np.array([1, 2, 2, 2])
(a[a == b]).shape[0]

3

In [ ]:
from sklearn.datasets import load_iris

data = load_iris()
X, y = data.data, data.target
exp = DecisionTree(X, y, 3)
y_predict = exp.output(20)
acc = (exp.y_test[y_predict == exp.y_test]).shape[0] / exp.y_test.shape[0]
print('Accuracy = ', acc)


Accuracy =  0.5526315789473685
